In [ ]:
import os
import sys
import gensim
import pandas as pd
from gensim.models.doc2vec import LabeledSentence
from tqdm import tqdm
from sklearn import utils
from gensim.models import Doc2Vec
import multiprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

In [ ]:
# all_corpus = pd.read_csv('./corpus/prosa/data_clean_punctuation/all_data.csv')
all_corpus = pd.read_csv('./corpus/tripadvisor/corpus_formalized_cleaned.csv')
all_corpus.head()

In [ ]:
def labelize_text(text,label):
    result = []
    prefix = label
    for i, t in zip(text.index, text):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result
  
all_x = all_corpus['content']
all_x_w2v = labelize_text(all_x, 'ALL')

In [ ]:
cores = multiprocessing.cpu_count()
model_dbow = Doc2Vec(dm=0, size=100, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_x_w2v)])
model_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)

In [ ]:
cores = multiprocessing.cpu_count()
model_dmc = Doc2Vec(dm=1, dm_concat=1, size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_dmc.build_vocab([x for x in tqdm(all_x_w2v)])
model_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)

In [ ]:
cores = multiprocessing.cpu_count()
model_dmm = Doc2Vec(dm=1, dm_mean=1, size=100, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_dmm.build_vocab([x for x in tqdm(all_x_w2v)])
model_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)

In [ ]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in all_x_w2v])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

In [ ]:
# with open('./vectorizer/prosa/tfidf.pickle', 'wb') as fin:
with open('./vectorizer/tripadvisor/tfidf.pickle', 'wb') as fin:
    pickle.dump(tfidf, fin)

In [ ]:
tfidf = pickle.load(open('./vectorizer/prosa/tfidf.pickle', 'rb'))
# tfidf = pickle.load(open('./vectorizer/tripadvisor/tfidf.pickle', 'rb'))

In [ ]:
# model_dbow.save("./vectorizer/prosa/model_dbow.model")
# model_dmc.save("./vectorizer/prosa/model_dmc.model")
# model_dmm.save("./vectorizer/prosa/model_dmm.model")
# model_dbow.save("./vectorizer/tripadvisor/model_dbow.model")
# model_dmc.save("./vectorizer/tripadvisor/model_dmc.model")
model_dmm.save("./vectorizer/tripadvisor/model_dmm.model")